# Dataset

In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
required_libs = [ ("numpy", "numpy"),
                 ("pandas", "pandas"),
                 ("seaborn", "seaborn"),
                 ("matplotlib", "matplotlib"),
                 ("sklearn", "sklearn"),
                 ("colorama", "colorama"),
                 ("emoji", "emoji"),
                 ("catboost", "catboost")
                ]

In [10]:
def is_lib_exists(name):
    import importlib
    lib = importlib.util.find_spec(name)
    return lib is not None

In [11]:
for (clz,lib) in required_libs:
    if not is_lib_exists(clz):
        print(f"Installing {lib}")
        !pip -qq install {lib}
    else:
        print(f"{lib} exists")

numpy exists
pandas exists
seaborn exists
matplotlib exists
sklearn exists
colorama exists
emoji exists
Installing catboost
     |████████████████████████████████| 66.1MB 62kB/s 


In [12]:
# Uninstall fastai ver 1 and install ver 2
!pip uninstall -qq --yes fastai 
!pip install -q fastai
!pip install -q nbdev
import fastai
print(fastai.__version__)

2.0.13


In [13]:
COLAB = True

DATASET_NAME = '4D.zip'

FEATURE_DATASET_PREFIX = 'feature_matrix_d2_v3'

In [53]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
import traceback
import catboost
import gc
from pathlib import Path
from dateutil.relativedelta import *
from datetime import *
from catboost import *
from catboost import datasets
from catboost import CatBoostClassifier
from scipy import stats
from scipy.stats.stats import pearsonr
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.metrics import roc_auc_score
np.set_printoptions(precision=4)
pd.options.display.max_columns = None

In [15]:
%matplotlib inline
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [16]:
from IPython.display import display

In [17]:
# check catboost version
print(catboost.__version__)
!python --version

0.24.1
Python 3.6.9


In [18]:
# colab setup
if COLAB:
  !rm -rf dl-projects
  !git clone https://github.com/mengwangk/dl-projects
  
  !cp dl-projects/utils* .
  !cp dl-projects/preprocess* .
  !cp dl-projects/plot* .
  
  from google.colab import drive
  drive.mount('/content/gdrive')
  GDRIVE_DATASET_FOLDER = Path('gdrive/My Drive/datasets/')
  DATASET_PATH = GDRIVE_DATASET_FOLDER
  ORIGIN_DATASET_PATH = Path('dl-projects/datasets')
  #!ls -l gdrive/"My Drive"/datasets/ --block-size=M

DATASET = DATASET_PATH/f"{FEATURE_DATASET_PREFIX}.ft"
ORIGIN_DATASET = ORIGIN_DATASET_PATH/DATASET_NAME

Cloning into 'dl-projects'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 2287 (delta 46), reused 23 (delta 11), pack-reused 2210
Receiving objects: 100% (2287/2287), 80.31 MiB | 13.53 MiB/s, done.
Resolving deltas: 100% (1425/1425), done.
Mounted at /content/gdrive


## EDA

In [19]:
from preprocess import *
from utils import feature_selection, plot_feature_importances
from plot import plot_correlation_matrix, plot_labeled_scatter

In [20]:
data = pd.read_feather(DATASET)
origin_data = format_tabular(ORIGIN_DATASET)

In [21]:
jan_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jan.ft")
feb_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_feb.ft")
mar_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_mar.ft")
apr_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_apr.ft")
may_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_may.ft")
jun_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jun.ft")
jul_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jul.ft")

In [22]:
new_data = data.append(jan_2020[data.columns],ignore_index=True)
new_data = new_data.append(feb_2020[data.columns],ignore_index=True)
new_data = new_data.append(mar_2020[data.columns],ignore_index=True)
new_data = new_data.append(apr_2020[data.columns],ignore_index=True)
new_data = new_data.append(may_2020[data.columns],ignore_index=True)
new_data = new_data.append(jun_2020[data.columns],ignore_index=True)
new_data = new_data.append(jul_2020[data.columns],ignore_index=True)
data.shape, new_data.shape 

((959893, 217), (1029893, 217))

In [23]:
data = new_data

In [24]:
data.isna().sum().sort_values(ascending=False)

CUM_SUM(SKEW(Results.TotalStrike))           7685
CUM_MEAN(TREND(Results.DrawNo, DrawDate))    7685
TREND(Results.CUM_SUM(DrawNo), DrawDate)     7685
TREND(Results.CUM_SUM(LuckyNo), DrawDate)    7685
CUM_SUM(SKEW(Results.LuckyNo))               7685
                                             ... 
CUM_SUM(MIN(Results.DrawNo))                    0
NUM_UNIQUE(Results.DAY(DrawDate))               0
NUM_UNIQUE(Results.MONTH(DrawDate))             0
SUM(Results.PERCENTILE(LuckyNo))                0
NumberId                                        0
Length: 217, dtype: int64

In [25]:
# data.info(max_cols=500, null_counts=True)
# data.columns.tolist()

In [26]:
feature_matrix = data

### fastai to handle missing values
# feature_matrix = data.fillna(0)
# feature_matrix.sort_values(by=['time', 'MAX(Results.LuckyNo)'], inplace=True)
# feature_matrix.head(20)

In [27]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, chi2, f_regression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

# other_features = ['Label', 'NumberId', 'time', 'TotalStrike',  'month', 'year']
# feature_matrix = feature_selection(data)

def select_features(df, func=f_regression):
  X = df.drop(columns=other_features)
  y = df.Label
  fs = SelectKBest(score_func=func, k=50)
  X_selected = fs.fit_transform(X, y)
  mask = fs.get_support()
  return X.columns[mask]

def reduce_features(df, lower_threshold=10, upper_threshold=10000):
  cols_to_drop = []
  for col in df.columns:
    if df[col].nunique() > upper_threshold or df[col].nunique() < lower_threshold:
      cols_to_drop.append(col)
  print(cols_to_drop)
  return cols_to_drop, df.drop(columns=cols_to_drop)

def select_by_variance(df, threshold=0):
  """Select features by variance"""
  filter = VarianceThreshold(threshold=threshold)
  filter.fit(df)
  cols_to_drop = [column for column in df.columns
                  if column not in df.columns[filter.get_support()]]
  print(f"No of columns to drop - {len(cols_to_drop)}")
  print(cols_to_drop)
  return cols_to_drop

def select_duplicates(df):
  df_transposed = df.head(20000).T
  print(df_transposed.duplicated().sum())
  unique_features = df_transposed.drop_duplicates(keep='first').T
  print(unique_features.shape)
  duplicated_features = [dup_col for dup_col in df.columns if dup_col not in unique_features.columns]
  return duplicated_features

def select_correlated(df):
  correlated_features = set()
  correlation_matrix = df.corr()
  for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
  print(len(correlated_features))
  print(correlated_features)
  return correlated_features


In [28]:
selected_features = feature_matrix.columns.to_list()
# display(len(selected_features), selected_features)
print(len(selected_features))

217


In [29]:
df_selected_features = feature_matrix[selected_features]

In [30]:
# def split_data(df, dt):
#   y_train = df.loc[df['time'] < dt, 'Label']
#   y_validation = df.loc[df['time'] == dt, 'Label']
# 
#   X_train = df[df['time'] < dt].drop(columns = ['NumberId', 'time', 'Label', 'TotalStrike', 'year', 'month'])
#   X_validation = df[df['time'] == dt].drop(columns = ['NumberId', 'time', 'Label', 'TotalStrike', 'year', 'month'])
# 
#   # Reduce by standard deviation
#   # X_train = X_train.loc[:, X_train.std() > .0]
#   # X_validation = X_validation[X_train.columns]
# 
#   return X_train, X_validation, y_train, y_validation

In [31]:
TARGET_DT = datetime(2020,1,1)

# X_train, X_validation, y_train, y_validation = split_data(df_selected_features, TARGET_MONTH)
# X_train.shape, X_validation.shape, y_train.shape, y_validation.shape

In [32]:
# cols_to_drop = select_by_variance(X_train,0.0)
# X_train = X_train.drop(columns=cols_to_drop)
# X_validation = X_validation[X_train.columns]
# X_train.shape, X_validation.shape
# 
# cols_to_drop = select_by_variance(X_train,0.01)
# X_train = X_train.drop(columns=cols_to_drop)
# X_validation = X_validation[X_train.columns]
# X_train.shape, X_validation.shape
# 
# cols_to_drop = select_duplicates(X_train)
# X_train = X_train.drop(columns=cols_to_drop)
# X_validation = X_validation[X_train.columns]
# X_train.shape, X_validation.shape
# 
# cols_to_drop = select_correlated(X_train)
# X_train = X_train.drop(columns=cols_to_drop)
# X_validation = X_validation[X_train.columns]
# X_train.shape, X_validation.shape

In [33]:
# CAT_FEATURES = []
# UNIQUE_THRESHOLD = 500
# for col in X_train.select_dtypes(include=['int64']).columns:
#   print(f"{col}: Index - {X_train.columns.get_loc(col)}, Unique values - {X_train[col].nunique()}")
#   if X_train[col].nunique() <= UNIQUE_THRESHOLD:
#     CAT_FEATURES.append(X_train.columns.get_loc(col))
#   
# print(CAT_FEATURES)

In [34]:
df_selected_features.columns

Index(['NumberId', 'time', 'STD(Results.DrawNo)', 'STD(Results.TotalStrike)',
       'STD(Results.LuckyNo)', 'MAX(Results.DrawNo)',
       'MAX(Results.TotalStrike)', 'MAX(Results.LuckyNo)',
       'MIN(Results.DrawNo)', 'MIN(Results.TotalStrike)',
       ...
       'PERCENTILE(SUM(Results.DrawNo))',
       'PERCENTILE(TREND(Results.DrawNo, DrawDate))',
       'PERCENTILE(LAST(Results.LuckyNo))', 'PERCENTILE(MAX(Results.LuckyNo))',
       'TotalStrike', 'Label', 'MODE(Results.PrizeType)_Prize',
       'LAST(Results.PrizeType)_Prize', 'month', 'year'],
      dtype='object', length=217)

In [35]:
cols_to_drop = select_by_variance(df_selected_features.drop(columns=['Label', 'NumberId', 'time', 'TotalStrike',  'month', 'year']), 0.01)

No of columns to drop - 68
['STD(Results.TotalStrike)', 'STD(Results.LuckyNo)', 'MAX(Results.TotalStrike)', 'MIN(Results.TotalStrike)', 'MEAN(Results.TotalStrike)', 'SKEW(Results.TotalStrike)', 'SKEW(Results.LuckyNo)', 'LAST(Results.TotalStrike)', 'TREND(Results.TotalStrike, DrawDate)', 'TREND(Results.LuckyNo, DrawDate)', 'TREND(Results.DrawNo, DrawDate)', 'NUM_UNIQUE(Results.PrizeType)', 'STD(Results.PERCENTILE(DrawNo))', 'STD(Results.PERCENTILE(TotalStrike))', 'STD(Results.CUM_MEAN(TotalStrike))', 'STD(Results.PERCENTILE(LuckyNo))', 'MAX(Results.PERCENTILE(TotalStrike))', 'MAX(Results.CUM_MEAN(TotalStrike))', 'MIN(Results.PERCENTILE(TotalStrike))', 'MIN(Results.CUM_MEAN(TotalStrike))', 'MEAN(Results.PERCENTILE(TotalStrike))', 'MEAN(Results.CUM_MEAN(TotalStrike))', 'SKEW(Results.PERCENTILE(TotalStrike))', 'SKEW(Results.CUM_MEAN(TotalStrike))', 'SKEW(Results.PERCENTILE(LuckyNo))', 'LAST(Results.PERCENTILE(TotalStrike))', 'LAST(Results.CUM_MEAN(TotalStrike))', 'TREND(Results.CUM_MEAN(Lu

In [36]:
# Prepare the data
# unused_features = ['NumberId', 'time', 'TotalStrike',  'month', 'year']
unused_features = ['time', 'TotalStrike'] + cols_to_drop
print("Columns to drop ", unused_features)
df = df_selected_features.drop(columns=unused_features)

# Target label. No need to np.log
dep_var = 'Label'

# Train test split
procs = [Categorify, FillMissing, Normalize]
cond = (df.year < TARGET_DT.year) | (df.month < TARGET_DT.month)
train_idx = np.where( cond)[0]
valid_idx = np.where(~cond)[0]
splits = (list(train_idx),list(valid_idx))

Columns to drop  ['time', 'TotalStrike', 'STD(Results.TotalStrike)', 'STD(Results.LuckyNo)', 'MAX(Results.TotalStrike)', 'MIN(Results.TotalStrike)', 'MEAN(Results.TotalStrike)', 'SKEW(Results.TotalStrike)', 'SKEW(Results.LuckyNo)', 'LAST(Results.TotalStrike)', 'TREND(Results.TotalStrike, DrawDate)', 'TREND(Results.LuckyNo, DrawDate)', 'TREND(Results.DrawNo, DrawDate)', 'NUM_UNIQUE(Results.PrizeType)', 'STD(Results.PERCENTILE(DrawNo))', 'STD(Results.PERCENTILE(TotalStrike))', 'STD(Results.CUM_MEAN(TotalStrike))', 'STD(Results.PERCENTILE(LuckyNo))', 'MAX(Results.PERCENTILE(TotalStrike))', 'MAX(Results.CUM_MEAN(TotalStrike))', 'MIN(Results.PERCENTILE(TotalStrike))', 'MIN(Results.CUM_MEAN(TotalStrike))', 'MEAN(Results.PERCENTILE(TotalStrike))', 'MEAN(Results.CUM_MEAN(TotalStrike))', 'SKEW(Results.PERCENTILE(TotalStrike))', 'SKEW(Results.CUM_MEAN(TotalStrike))', 'SKEW(Results.PERCENTILE(LuckyNo))', 'LAST(Results.PERCENTILE(TotalStrike))', 'LAST(Results.CUM_MEAN(TotalStrike))', 'TREND(Result

In [37]:
# Continuous and categorical variables
cont,cat = cont_cat_split(df, max_card=9000, dep_var=dep_var)
print("Continous features: ", cont)
print("Categorical features:", cat)

Continous features:  ['NumberId', 'STD(Results.DrawNo)', 'MAX(Results.LuckyNo)', 'MIN(Results.LuckyNo)', 'MEAN(Results.DrawNo)', 'MEAN(Results.LuckyNo)', 'SKEW(Results.DrawNo)', 'LAST(Results.results_index)', 'LAST(Results.LuckyNo)', 'AVG_TIME_BETWEEN(Results.DrawDate)', 'SUM(Results.DrawNo)', 'SUM(Results.LuckyNo)', 'TIME_SINCE(first_Results_time)', 'TIME_SINCE_PREVIOUS(first_Results_time)', 'STD(Results.TIME_SINCE(DrawDate))', 'STD(Results.CUM_MEAN(DrawNo))', 'STD(Results.CUM_SUM(LuckyNo))', 'STD(Results.TIME_SINCE_PREVIOUS(DrawDate))', 'STD(Results.CUM_SUM(TotalStrike))', 'STD(Results.CUM_SUM(DrawNo))', 'STD(Results.CUM_MEAN(LuckyNo))', 'MAX(Results.TIME_SINCE(DrawDate))', 'MAX(Results.CUM_MEAN(DrawNo))', 'MAX(Results.CUM_SUM(LuckyNo))', 'MAX(Results.PERCENTILE(DrawNo))', 'MAX(Results.TIME_SINCE_PREVIOUS(DrawDate))', 'MAX(Results.CUM_SUM(TotalStrike))', 'MAX(Results.CUM_SUM(DrawNo))', 'MAX(Results.CUM_MEAN(LuckyNo))', 'MAX(Results.PERCENTILE(LuckyNo))', 'MIN(Results.TIME_SINCE(DrawD

In [38]:
gc.collect() # Garbage collect

to = TabularPandas(df, procs=procs, cat_names=cat, cont_names=cont, y_names=dep_var, splits=splits)
len(to.train),len(to.valid)

4

(959893, 70000)

In [39]:
dls = to.dataloaders(1024)

In [40]:
y = to.train.y
y.min(),y.max()

(0, 1)

In [41]:
doc(tabular_learner)

In [54]:
learn = tabular_learner(dls, layers=[500,250], n_out=1, metrics=[accuracy, roc_auc_score], loss_func=torch.nn.BCELoss)

In [55]:
learn.lr_find()

RuntimeError: ignored

In [44]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.033224,0.015095,0.985843,03:26
1,0.033012,0.014206,0.985843,03:26
2,0.031215,0.014174,0.985843,03:28
3,0.032212,0.014437,0.985843,03:25
4,0.030448,0.014842,0.985843,03:24


In [45]:
preds,targs = learn.get_preds()
print(len(preds), len(targs))

70000 70000


In [46]:
print(preds)

tensor([[0.0456],
        [0.0404],
        [0.0067],
        ...,
        [0.0880],
        [0.0378],
        [0.0611]])


In [47]:
print(targs)

tensor([[1],
        [1],
        [1],
        ...,
        [0],
        [0],
        [0]], dtype=torch.int8)
